Calculated using Linear Regression and Gradient Boosting Regressor

In [2]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train.csv')

In [3]:
# create month and year instead of full date
df['year'] = df.timestamp.apply(lambda x: pd.to_datetime(x).year - 2000)
df['month'] = df.timestamp.apply(lambda x: pd.to_datetime(x).month)
df.drop(['timestamp'],axis=1,inplace=True)

In [4]:
# rearranging the columns
cols = df.columns.tolist()
cols = cols[0:1] + cols[-2:] + cols[1:-2]
df = df[cols]

In [5]:
# create a list with column values
cols = []
for i,val in enumerate(df):
    cols.append(val)

In [56]:
# adjusting some more year values
df.set_value(10089,'build_year',2005)
df.set_value(10090,'build_year',2009)
df.set_value(29767,'build_year',2008)
df.set_value(30150,'build_year',2015)
df.set_value(13992,'build_year',2000)
df.set_value(30275,'build_year',1971)
df.set_value(15220,'build_year',1965)
df.set_value(11271,'build_year',2003)
df.set_value(19127,'build_year',2003)
df.head()

,id,year,month,full_sq,life_sq,floor,max_floor,material,build_year,num_room,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,11,8,43,27.0,4.0,9.0,1.0,3.0,1.0,...,9,4,0,13,22,1,0,52,4,5850000
1,2,11,8,34,19.0,3.0,9.0,1.0,3.0,1.0,...,15,3,0,15,29,1,10,66,14,6000000
2,3,11,8,43,29.0,2.0,8.0,1.0,3.0,1.0,...,10,3,0,11,27,0,4,67,10,5700000
3,4,11,9,89,50.0,9.0,13.0,1.0,4.0,3.0,...,11,2,1,4,4,0,0,26,3,13100000
4,5,11,9,77,77.0,4.0,10.0,1.0,3.0,2.0,...,319,108,17,135,236,2,91,195,14,16331452


In [7]:
for i in df.index:
    if df.iloc[i]['build_year'] == 1:
        df.set_value(i,'build_year',float('NaN'))

In [12]:
# we've now got the years as categorical values
# all - 1920 - 1
# 1921 - 1945 - 2
# 1946 - 1970 - 3
# 1971 - 1995 - 4
# 1995 - present - 5

df.set_value((df.build_year <= 1920) & (df.build_year > 1000),'build_year', 1)
df.set_value((df.build_year <= 1945) & (df.build_year > 1920),'build_year', 2)
df.set_value((df.build_year <= 1970) & (df.build_year > 1945),'build_year', 3)
df.set_value((df.build_year <= 1995) & (df.build_year > 1970),'build_year', 4)
df.set_value((df.build_year > 1995),'build_year', 5)
df.head()

,id,year,month,full_sq,life_sq,floor,max_floor,material,build_year,num_room,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,11,8,43,27.0,4.0,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,11,8,34,19.0,3.0,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,11,8,43,29.0,2.0,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,11,9,89,50.0,9.0,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,11,9,77,77.0,4.0,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [13]:
# some values from life_sq are greater than full_sq; that shouldn't be the case
ind = df[df['life_sq'] > df['full_sq']].index

for i in ind:
    df.set_value(i,'life_sq',df.iloc[i]['life_sq']/10)

In [14]:
# few more remaining, will rerun the above and set one of them manually
df[df['life_sq'] > df['full_sq']]

,id,year,month,full_sq,life_sq,floor,max_floor,material,build_year,num_room,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
11332,11335,13,10,1,4.0,10.0,17.0,1.0,5.0,1.0,...,1,0,0,1,5,1,0,14,1,3357774
13546,13549,13,12,79,747.8,8.0,17.0,1.0,5.0,3.0,...,0,0,0,1,7,1,0,12,1,7705000
22412,22415,14,9,1,4.7,11.0,17.0,1.0,5.0,1.0,...,278,118,24,68,120,0,73,167,9,14923206
24296,24299,14,11,0,7.7,4.0,17.0,1.0,0.0,3.0,...,1,1,0,2,12,0,1,10,0,8167508
26264,26267,14,12,1,6.0,17.0,17.0,1.0,5.0,2.0,...,1,0,0,4,5,0,0,4,1,5983900
26363,26366,14,12,1,6.4,22.0,22.0,1.0,NaN,2.0,...,7,1,0,7,16,0,0,34,5,14430018


In [15]:
ind = df[df['life_sq'] > df['full_sq']].index

for i in ind:
    df.set_value(i,'life_sq',df.iloc[i]['life_sq']/10)

In [17]:
df.set_value(24296,'life_sq',0)
df.tail()

,id,year,month,full_sq,life_sq,floor,max_floor,material,build_year,num_room,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
30466,30469,15,6,44,27.0,7.0,9.0,1.0,4.0,2.0,...,15,5,0,15,26,1,2,84,6,7400000
30467,30470,15,6,86,59.0,3.0,9.0,2.0,2.0,4.0,...,313,128,24,98,182,1,82,171,15,25000000
30468,30471,15,6,45,NaN,10.0,20.0,1.0,NaN,1.0,...,1,1,0,2,12,0,1,11,1,6970959
30469,30472,15,6,64,32.0,5.0,15.0,1.0,5.0,2.0,...,22,1,1,6,31,1,4,65,7,13500000
30470,30473,15,6,43,28.0,1.0,9.0,1.0,3.0,2.0,...,5,2,0,7,16,0,9,54,10,5600000


In [18]:
def fill(main_df, df_col):
    """
    Function to fill in the empty spaces in a particular column
    
    main_df = section of the main data frame from start to the column of interest
    
    df_col = the column to be filled
    """
    # creating sets for linear regression
    full_df = main_df[~main_df[df_col].isnull()]
    X = full_df[full_df.columns[1:-1]]
    y = full_df[full_df.columns[-1]]
    
    # for prediction
    ln = len(main_df.columns)
    jk = cols[1:(ln-1)]
    to_pred = main_df[main_df[df_col].isnull()][jk]
    
    # converting them to np arrays and reshaping
    s = int(X.shape[1])
    X = np.asarray(X.values.reshape(-len(X),s))
    y = np.asarray(y.values.reshape(-1,1))
    
    # linear regression training
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    reg = LinearRegression()
    reg.fit(X_train,y_train)
    t_score = reg.score(X_train, y_train)
    tt_score = reg.score(X_test,y_test)
    
    # fitting new values
    pred = reg.predict(to_pred)
    
    # plugging in the new values
    count = 0
    for i in main_df.index:
        if np.isnan(main_df.iloc[i][df_col]):                                     
            main_df.set_value(i,df_col,int(pred[count]))
            count += 1
    
    return main_df

In [19]:
def check_corr(df, df_col):
    """
    Function to return the mini df for predicting the empty spaces.
    """
    cl = []
    for val in cols:
        if val == df_col:
            cl.append(val)
            break
        else:
            cl.append(val)
    dfr = df[cl]
    return dfr

In [20]:
# create new copy for df
df_c = df.copy()

In [21]:
# get the text columns
txt = []
for i in df:
    found = 0
    for j in df.index[0:10]:
        if isinstance(df[i][j], basestring):
            found = 1
    if found == 1:
        txt.append(i)

txt

['product_type',
 'sub_area',
 'culture_objects_top_25',
 'thermal_power_plant_raion',
 'incineration_raion',
 'oil_chemistry_raion',
 'radiation_raion',
 'railroad_terminal_raion',
 'big_market_raion',
 'nuclear_reactor_raion',
 'detention_facility_raion',
 'water_1line',
 'big_road1_1line',
 'railroad_1line',
 'ecology']

In [22]:
# product_type
# OwnerOccupier = 0
# Investment = 1
for i in df.index:
    if df.iloc[i]['product_type'] == 'Investment':
        df.set_value(i,'product_type',1)
    else:
        df.set_value(i,'product_type',0)

In [23]:
# sub_area
# 146 categories
# classified 0 to 145 based on the occurences, 0 being the lowest and 145 being the higest
# try with this reverted as well
areas = df.sub_area.value_counts().keys().tolist()
rng = range(0,146)
rng = rng[::-1]

In [24]:
# this takes a while to run - making a checkpoint right after
for i in df.index:
    for j in xrange(len(areas)):
        if df.iloc[i]['sub_area'] == areas[j]:
            df.set_value(i,'sub_area',rng[j])

In [25]:
# checkpoint
# df.to_csv('checkpoint.csv')

# importing checkpoint to save time
df = pd.read_csv('checkpoint.csv')

In [26]:
# sorting out the yes/no columns
yn = ['culture_objects_top_25',
 'thermal_power_plant_raion',
 'incineration_raion',
 'oil_chemistry_raion',
 'radiation_raion',
 'railroad_terminal_raion',
 'big_market_raion',
 'nuclear_reactor_raion',
 'detention_facility_raion',
 'water_1line',
 'big_road1_1line',
 'railroad_1line']

In [27]:
for i in df.index:
    for j in yn:
        if df.iloc[i][j] == 'yes':
            df.set_value(i,j,1)
        else:
            df.set_value(i,j,0)

In [28]:
# last one with text, ecology

In [29]:
rtl = list(df.ecology.value_counts().keys())
rtl

['poor', 'no data', 'good', 'excellent', 'satisfactory']

In [30]:
# setting the ratings in the order of text (yes, i will be lazy on this one)
rate = [1,0,4,5,3]

In [31]:
for i in df.index:
    for ind,j in enumerate(rtl):
        if df.iloc[i]['ecology'] == j:
            df.set_value(i,'ecology',rate[ind])

In [32]:
# check columns with empty fields
# this takes a while as well - once columns are filled set another checkpoint
empty = []
for i in df:
    found = 0
    for j in df.index:
        if np.isnan(df.iloc[j][i]):
            found = 1
            break
    if found == 1:
        empty.append(i)

In [33]:
empty_vals = df[empty]

In [34]:
# empty values checkpoint
# empty_vals.to_csv('empty.csv')

# importing empty values columns to save time
empty_vals = pd.read_csv('empty.csv')
empty = list(empty_vals.columns)

In [112]:
# sort out empty columns by applying linear regression on them
# for i in empty:
#     # get the df section
#     main_df = check_corr(df, i)

#     # fill it up
#     new_df = fill(main_df,i)

#     # reassign
#     for j in new_df:
#         df[i] = new_df[i]

In [ ]:
# fill in empty year columns
# get the df section in place
main_df = check_corr(df, 'build_year')

# fill it up
new_df = fill(main_df,'build_year')

# assign it
df['build_year'] = new_df['build_year']

In [35]:
# redoing the emtpy spaces as they were based on the wrong years
df[empty] = df_c[empty]

In [36]:
# sort out empty columns by applying linear regression on them
for i in empty:
    # get the df section
    main_df = check_corr(df, i)

    # fill it up
    new_df = fill(main_df,i)

    # reassign
    for j in new_df:
        df[i] = new_df[i]

In [100]:
# save checkpoint
# df.to_csv('df_fix.csv')

# importing checkpoint to save time
df = pd.read_csv('df_fix.csv')
# dropping the additional column added
df.drop('Unnamed: 0',axis=1,inplace=True)

In [129]:
# setting X and y
fx = df.columns.tolist()[1:-1]
X = df[fx]
y = df[df.columns.tolist()[-1]]

In [130]:
X = np.array(X.values.reshape(-len(X),X.shape[1]))
y = np.array(y.values.reshape(-1,1))

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [131]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [42]:
cls = LinearRegression()

In [43]:
cls.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [44]:
cls.score(X_train,y_train)

0.48278417976564847

In [45]:
cls.score(X_test,y_test)

0.47993396860789628

In [46]:
test = pd.read_csv('test.csv')

In [47]:
# refine the test set to be the same format
test.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,30474,2015-07-01,39.0,20.7,2,9,1,1998.0,1,8.9,...,8,0,0,0,1,10,1,0,14,1
1,30475,2015-07-01,79.2,NaN,8,17,1,0.0,3,1.0,...,4,1,1,0,2,11,0,1,12,1
2,30476,2015-07-01,40.5,25.1,3,5,2,1960.0,2,4.8,...,42,11,4,0,10,21,0,10,71,11
3,30477,2015-07-01,62.8,36.0,17,17,1,2016.0,2,62.8,...,1,1,2,0,0,10,0,0,2,0
4,30478,2015-07-01,40.0,40.0,17,17,1,0.0,1,1.0,...,5,1,1,0,2,12,0,1,11,1


In [48]:
# create month and year instead of full date
test['year'] = test.timestamp.apply(lambda x: pd.to_datetime(x).year - 2000)
test['month'] = test.timestamp.apply(lambda x: pd.to_datetime(x).month)
test.drop(['timestamp'],axis=1,inplace=True)

In [49]:
# rearranging the columns
cols = test.columns.tolist()
cols = cols[0:1] + cols[-2:] + cols[1:-2]
test = test[cols]

In [51]:
# fixing some years
test.set_value(2995,'build_year',2015)
test.set_value(2010,'build_year',2000)
test.head()

,id,year,month,full_sq,life_sq,floor,max_floor,material,build_year,num_room,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,30474,15,7,39.0,20.7,2,9,1,1998.0,1,...,8,0,0,0,1,10,1,0,14,1
1,30475,15,7,79.2,NaN,8,17,1,0.0,3,...,4,1,1,0,2,11,0,1,12,1
2,30476,15,7,40.5,25.1,3,5,2,1960.0,2,...,42,11,4,0,10,21,0,10,71,11
3,30477,15,7,62.8,36.0,17,17,1,2016.0,2,...,1,1,2,0,0,10,0,0,2,0
4,30478,15,7,40.0,40.0,17,17,1,0.0,1,...,5,1,1,0,2,12,0,1,11,1


In [52]:
for i in test.index:
    if test.iloc[i]['build_year'] == 1:
        test.set_value(i,'build_year',float('NaN'))

In [55]:
test.groupby('build_year').sum().index.tolist()[:10]

[0.0, 1875.0, 1890.0, 1895.0, 1898.0, 1902.0, 1905.0, 1906.0, 1908.0, 1910.0]

In [59]:
# all - 1920 - 1
# 1921 - 1945 - 2
# 1946 - 1970 - 3
# 1971 - 1995 - 4
# 1995 - present - 5

test.set_value((test.build_year <= 1920) & (test.build_year > 1000),'build_year', 1)
test.set_value((test.build_year <= 1945) & (test.build_year > 1920),'build_year', 2)
test.set_value((test.build_year <= 1970) & (test.build_year > 1945),'build_year', 3)
test.set_value((test.build_year <= 1995) & (test.build_year > 1970),'build_year', 4)
test.set_value((test.build_year > 1995),'build_year', 5)
test.head()

,id,year,month,full_sq,life_sq,floor,max_floor,material,build_year,num_room,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,30474,15,7,39.0,20.7,2,9,1,5.0,1,...,8,0,0,0,1,10,1,0,14,1
1,30475,15,7,79.2,NaN,8,17,1,0.0,3,...,4,1,1,0,2,11,0,1,12,1
2,30476,15,7,40.5,25.1,3,5,2,3.0,2,...,42,11,4,0,10,21,0,10,71,11
3,30477,15,7,62.8,36.0,17,17,1,5.0,2,...,1,1,2,0,0,10,0,0,2,0
4,30478,15,7,40.0,40.0,17,17,1,0.0,1,...,5,1,1,0,2,12,0,1,11,1


In [60]:
test.build_year.value_counts()

5.0    2597
4.0    1819
3.0    1568
0.0     369
2.0      54
1.0      19
Name: build_year, dtype: int64

In [61]:
test.life_sq.isnull().value_counts()

False    6486
True     1176
Name: life_sq, dtype: int64

In [ ]:
# empty spaces in life_sq, got to fill 'em up
main_df = check_corr(test, 'life_sq')

# fill it up
new_df = fill(main_df,'life_sq')

# assign it
test['life_sq'] = new_df['life_sq']

In [ ]:
# fill up build_year column for test
# get the df section in place
main_df = check_corr(test, 'build_year')

# fill it up
new_df = fill(main_df,'build_year')

# assign it
test['build_year'] = new_df['build_year']

In [62]:
# get the text columns
txt = []
for i in test:
    found = 0
    for j in test.index[0:10]:
        if isinstance(test[i][j], basestring):
            found = 1
    if found == 1:
        txt.append(i)

txt

['product_type',
 'sub_area',
 'culture_objects_top_25',
 'thermal_power_plant_raion',
 'incineration_raion',
 'oil_chemistry_raion',
 'radiation_raion',
 'railroad_terminal_raion',
 'big_market_raion',
 'nuclear_reactor_raion',
 'detention_facility_raion',
 'water_1line',
 'big_road1_1line',
 'railroad_1line',
 'ecology']

In [63]:
# product_type
# OwnerOccupier = 0
# Investment = 1
for i in test.index:
    if test.iloc[i]['product_type'] == 'Investment':
        test.set_value(i,'product_type',1)
    else:
        test.set_value(i,'product_type',0)

In [74]:
# didn't memorise the way regions were organised initially
# going back to get the order, eliminate the extra item and assign the correct number to each region
sch = test.sub_area.values.tolist()
for i in areas:
    found = 0
    for j in sch:
        if i == j:
            found = 1
    if found == 0:
        print i

Poselenie Klenovskoe


In [76]:
xm = pd.DataFrame()
xm['id'] = rng
xm['vals'] = areas

In [77]:
xm[xm.vals == 'Poselenie Klenovskoe']

,id,vals
145,0,Poselenie Klenovskoe


In [79]:
xm.drop(145).tail()

,id,vals
140,5,Poselenie Marushkinskoe
141,4,Molzhaninovskoe
142,3,Poselenie Kievskij
143,2,Poselenie Shhapovskoe
144,1,Poselenie Mihajlovo-Jarcevskoe


In [81]:
# rearranged index values
orr = xm.sort_values('vals')['id'].values.tolist()

In [83]:
sch = test.sub_area.value_counts().keys().tolist()

In [85]:
# arranging those areas alphabetically
sch.sort()

In [86]:
# this takes a while to run - making a checkpoint right after
for i in test.index:
    for j in xrange(len(sch)):
        if test.iloc[i]['sub_area'] == sch[j]:
            test.set_value(i,'sub_area',orr[j])

In [87]:
# test.to_csv('checkpoint_test.csv')

# importing checkpoint to save time
test = pd.read_csv('checkpoint_test.csv')

In [88]:
# sorting out the yes/no columns
yn = ['culture_objects_top_25',
 'thermal_power_plant_raion',
 'incineration_raion',
 'oil_chemistry_raion',
 'radiation_raion',
 'railroad_terminal_raion',
 'big_market_raion',
 'nuclear_reactor_raion',
 'detention_facility_raion',
 'water_1line',
 'big_road1_1line',
 'railroad_1line']

In [89]:
for i in test.index:
    for j in yn:
        if test.iloc[i][j] == 'yes':
            test.set_value(i,j,1)
        else:
            test.set_value(i,j,0)

In [90]:
test.ecology.value_counts()
# if there are the same, don't change rtl and rate below - not changing

poor            2060
no data         1765
good            1611
excellent       1226
satisfactory    1000
Name: ecology, dtype: int64

In [91]:
# ecology column
# rtl = list(df.ecology.value_counts().keys())

In [92]:
# setting the ratings in the order of text
# rate = [1,0,4,5,3]

In [93]:
for i in test.index:
    for ind,j in enumerate(rtl):
        if test.iloc[i]['ecology'] == j:
            test.set_value(i,'ecology',rate[ind])

In [ ]:
# this takes a while as well - once columns are filled set another checkpoint
empty = []
for i in test:
    found = 0
    for j in test.index:
        if np.isnan(test.iloc[j][i]):
            found = 1
            break
    if found == 1:
        empty.append(i)

In [ ]:
# sort out empty columns by applying linear regression on them
for i in empty:
    # get the df section
    main_df = check_corr(test, i)

    # fill it up
    new_df = fill(main_df,i)

    # reassign
    for j in new_df:
        test[i] = new_df[i]

In [4]:
# save checkpoint
# test.to_csv('test_fix.csv')

# importing the processed test file to save time
test = pd.read_csv('test_fix.csv')
# dropping the additional column added
test.drop('Unnamed: 0',axis=1,inplace=True)

In [179]:
# setting it up - only if imported
fx = test.columns.tolist()[1:]
ts = test[fx]
ts = np.array(ts.values.reshape(-len(X),ts.shape[1]))

In [104]:
pred = cls.predict(ts)

In [177]:
# eliminating one set of brackets, wasn't showing up nicely in the csv
# only for the linear regression
# prd = []
# for i in pred:
#     if i[0] < 0:
#         x = i[0] * (-1)
#         prd.append(x)
#     else:
#         prd.append(i[0])

In [26]:
# Gradient Boosting - better score than above
from sklearn.ensemble import GradientBoostingRegressor

In [181]:
sg = GradientBoostingRegressor(n_estimators=200)

In [182]:
# using the X and y generated from df, not test
sg.fit(X,y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [183]:
pred = sg.predict(ts)

In [ ]:
# exporting the final result
import csv
with open('output_GB.csv','wb') as resultFile:
    wr = csv.writer(resultFile, dialect='excel')
    wr.writerow(pred)